## grab all MERRA2 fields for the bounding box +/- 5 degrees and write out 

To help us composite gridded quantities by distance from the CWV=55mm boundary contour, with a `groupby().mean()` operation, we need to construct a new distance field on the MERRA2 grid.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

# Open MERRA2 files and combine all the wanted fields 
### (2D hourly collections on goldsmr4) 

In [26]:
d2diag = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXINT')
d2flux = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXFLX')
d2slv = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXSLV')

var_diag = ['dqvdt_phy','dqvdt_dyn','dqvdt_ana','uflxqv','vflxqv','swnettoa']
var_flux = ['eflux','evap','hflux','preccon','precanv','prectot','prevtot','prectotcorr']
var_slv = [ 't2m','ts','t850','t500','q850','q500','slp','tqv','tqi','tql'] 

d2super = d2diag[ var_diag ].merge(d2flux[ var_flux ]).merge(d2slv[ var_slv ])
d2super

<xarray.Dataset>
Dimensions:      (time: 379848, lat: 361, lon: 576)
Coordinates:
  * time         (time) datetime64[ns] 1980-01-01T00:30:00 ... 2023-05-01T23:...
  * lat          (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 88.5 89.0 89.5 90.0
  * lon          (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
Data variables: (12/24)
    dqvdt_phy    (time, lat, lon) float32 ...
    dqvdt_dyn    (time, lat, lon) float32 ...
    dqvdt_ana    (time, lat, lon) float32 ...
    uflxqv       (time, lat, lon) float32 ...
    vflxqv       (time, lat, lon) float32 ...
    swnettoa     (time, lat, lon) float32 ...
    ...           ...
    q850         (time, lat, lon) float32 ...
    q500         (time, lat, lon) float32 ...
    slp          (time, lat, lon) float32 ...
    tqv          (time, lat, lon) float32 ...
    tqi          (time, lat, lon) float32 ...
    tql          (time, lat, lon) float32 ...
Attributes:
    title:        MERRA-2 tavg1_2d_int_Nx: 2d,1-Hourly,Time-Averaged,Single-L...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Wed May 24 22:31:48 GMT 2023 : imported by GrADS Data Serve...

In [22]:
#var_diag = ['dqvdt_phy','dqvdt_dyn','dqvdt_ana','uflxqv','vflxqv','swnettoa']
var_flux_min = ['prectot']
var_slv_min = ['tqv','disph','tropt'] # dummy ones to re-use as distance and direction

d2min = d2flux[ var_flux_min ].merge(d2slv[ var_slv_min ])\
        .rename({"disph":"distance"}).rename({"tropt":"dir_from_centroid"})

d2min

<xarray.Dataset>
Dimensions:            (time: 379848, lat: 361, lon: 576)
Coordinates:
  * time               (time) datetime64[ns] 1980-01-01T00:30:00 ... 2023-05-...
  * lat                (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon                (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
Data variables:
    prectot            (time, lat, lon) float32 ...
    tqv                (time, lat, lon) float32 ...
    distance           (time, lat, lon) float32 ...
    dir_from_centroid  (time, lat, lon) float32 ...
Attributes:
    title:        MERRA-2 tavg1_2d_flx_Nx: 2d,1-Hourly,Time-Averaged,Single-L...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Wed May 24 22:31:24 GMT 2023 : imported by GrADS Data Serve...

## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [3]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [4]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

In [14]:
eq7cases

,Unnamed: 0.1,Unnamed: 0,lasttime,duration,areatime,tqv_values,maxarea,filename,ymdh,firsttime,dur_days,yyyy,mm,coastlat
92,92,102,2018-11-20 11:00:00,7 days 00:00:00,1892.647086,55.0,21.949572,2018_11_20_11_lat4p616S.geojson,2018_11_20_11,2018-11-13 11:00:00,7,2018,11,-4.616
99,99,109,2018-11-16 21:00:00,7 days 01:00:00,2667.686153,55.0,74.513991,2018_11_16_21_lat8p412S.geojson,2018_11_16_21,2018-11-09 20:00:00,7,2018,11,-8.412
127,127,137,2018-10-22 15:00:00,8 days 01:00:00,5261.462676,55.0,113.820949,2018_10_22_15_lat9p781N.geojson,2018_10_22_15,2018-10-14 14:00:00,8,2018,10,9.781
412,412,422,2018-03-19 12:00:00,7 days 18:00:00,16255.056979,55.0,221.334618,2018_03_19_12_lat9p318S.geojson,2018_03_19_12,2018-03-11 18:00:00,7,2018,3,-9.318
663,663,673,2017-12-01 11:00:00,11 days 00:00:00,6534.053999,55.0,92.725686,2017_12_01_11_lat9p845S.geojson,2017_12_01_11,2017-11-20 11:00:00,11,2017,12,-9.845
853,853,863,2017-05-28 11:00:00,7 days 04:00:00,5690.601728,55.0,50.852737,2017_05_28_11_lat5p291S.geojson,2017_05_28_11,2017-05-21 07:00:00,7,2017,5,-5.291
934,934,944,2017-05-09 01:00:00,10 days 03:00:00,13759.444603,55.0,308.867756,2017_05_09_01_lat3p355S.geojson,2017_05_09_01,2017-04-28 22:00:00,10,2017,5,-3.355
935,935,945,2017-05-08 05:00:00,9 days 07:00:00,9695.018611,55.0,124.822389,2017_05_08_05_lat1p160N.geojson,2017_05_08_05,2017-04-28 22:00:00,9,2017,5,1.160
936,936,946,2017-05-06 20:00:00,7 days 22:00:00,8426.735231,55.0,124.822389,2017_05_06_20_lat3p790S.geojson,2017_05_06_20,2017-04-28 22:00:00,7,2017,5,-3.790
937,937,947,2017-05-06 20:00:00,7 days 22:00:00,8629.762052,55.0,124.822389,2017_05_06_20_lat0p918N.geojson,2017_05_06_20,2017-04-28 22:00:00,7,2017,5,0.918


In [44]:
eq7cases.sort_values('areatime').lasttime

92      2018-11-20 11:00:00
99      2018-11-16 21:00:00
2508    2014-04-12 21:00:00
127     2018-10-22 15:00:00
853     2017-05-28 11:00:00
2462    2014-05-11 15:00:00
2461    2014-05-11 18:00:00
663     2017-12-01 11:00:00
963     2017-04-29 11:00:00
2156    2015-03-21 09:00:00
2155    2015-03-21 17:00:00
938     2017-05-06 18:00:00
936     2017-05-06 20:00:00
937     2017-05-06 20:00:00
964     2017-04-29 11:00:00
1464    2016-04-10 23:00:00
935     2017-05-08 05:00:00
1421    2016-04-29 17:00:00
1420    2016-05-01 00:00:00
1419    2016-05-02 05:00:00
2504    2014-04-17 06:00:00
1219    2017-01-07 14:00:00
934     2017-05-09 01:00:00
2501    2014-04-19 22:00:00
1462    2016-04-15 08:00:00
412     2018-03-19 12:00:00
2049    2015-04-30 02:00:00
1936    2015-09-17 21:00:00
2047    2015-04-30 17:00:00
2050    2015-04-29 21:00:00
2046    2015-05-01 17:00:00
2045    2015-05-01 21:00:00
2044    2015-05-05 01:00:00
2043    2015-05-05 07:00:00
2042    2015-05-05 21:00:00
1122    2017-02-27 2

In [45]:
# SUPER CASE: 24 field downloads 
d2loop = d2super

for icase in range(len(eq7cases)): 

# Continue from where it last crashed/ended:
#    if(icase < 8): 
#        continue

# Select cases by small to large areatime
    case = eq7cases.sort_values('areatime').iloc[icase]
    filename = case.filename
    print(icase, ' case, areatime ',filename, case.areatime)

# grab geopandas item
    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = d2loop.sel( lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                     lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                     time=slice(gdf.time.min(),gdf.time.max())             )
    
    print('Writing outputs: '+filename)
    d2.to_netcdf('~/Box/VaporLakes/data/LAKEBYLAKE/MERRA2_2D/'+\
        filename[0:-8]+'.M2_many2Dfields.nc')


0  case, areatime  2018_11_20_11_lat4p616S.geojson 1892.647086291892
Writing outputs: 2018_11_20_11_lat4p616S.geojson
1  case, areatime  2018_11_16_21_lat8p412S.geojson 2667.6861528738714
Writing outputs: 2018_11_16_21_lat8p412S.geojson
2  case, areatime  2014_04_12_21_lat5p095N.geojson 5008.481114870546
Writing outputs: 2014_04_12_21_lat5p095N.geojson
3  case, areatime  2018_10_22_15_lat9p781N.geojson 5261.462676095129
Writing outputs: 2018_10_22_15_lat9p781N.geojson
4  case, areatime  2017_05_28_11_lat5p291S.geojson 5690.601727745032
Writing outputs: 2017_05_28_11_lat5p291S.geojson
5  case, areatime  2014_05_11_15_lat4p404N.geojson 6050.818575241469
Writing outputs: 2014_05_11_15_lat4p404N.geojson
6  case, areatime  2014_05_11_18_lat2p131S.geojson 6060.850142585366
Writing outputs: 2014_05_11_18_lat2p131S.geojson
7  case, areatime  2017_12_01_11_lat9p845S.geojson 6534.053998839994
Writing outputs: 2017_12_01_11_lat9p845S.geojson
8  case, areatime  2017_04_29_11_lat7p001S.geojson 7493

In [27]:
!ls /Users/brianmapes/Box/VaporLakes/data/LAKEBYLAKE/MERRA2_2D/

2014_01_28_05_lat11p47S.M2_all2Dfields.nc
2014_04_12_21_lat5p095N.M2_all2Dfields.nc
2015_03_21_17_lat5p987N.M2_all2Dfields.nc
2015_09_17_21_lat3p507N.M2_all2Dfields.nc
2017_05_08_05_lat1p160N.M2_all2Dfields.nc
2017_05_09_01_lat3p355S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst.nc
2017_12_01_11_lat9p845S.M2_all2Dfields.nc
2018_03_19_12_lat9p318S.M2_all2Dfields.nc
2018_10_22_15_lat9p781N.M2_all2Dfields.nc
2018_11_16_21_lat8p412S.M2_all2Dfields.nc
2018_11_20_11_lat4p616S.M2_all2Dfields.nc
MINIMAL


In [46]:
!date

Sat Jun 17 01:08:19 MDT 2023


# Consider making IDV bundle for it 
### by relacing filename inside .xidv of a template bundle

In [21]:
!ls ../../Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/

2017_05_06_20_lat3p790S.M2_all2Dfields.nc
2017_05_08_05_lat1p160N.M2_all2Dfields.nc
2017_05_09_01_lat3p355S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.IMERG_30minute.nc
2017_05_28_11_lat5p291S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst.nc
2017_05_28_11_lat5p291S.M2_composite.nc
2017_12_01_11_lat9p845S.M2_all2Dfields.nc
2018_03_19_12_lat9p318S.M2_all2Dfields.nc
2018_10_22_15_lat9p781N.M2_all2Dfields.nc
2018_11_16_21_lat8p412S.M2_all2Dfields.nc
2018_11_20_11_lat4p616S.M2_all2Dfields.nc
direction_field_wrong_fixable


In [32]:
# inplace replacement works 

!cp ../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv test.txt

!sed -i '' "s/2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst/2018_11_20_11_lat4p616S.M2_all2Dfields.nc]/g" test.txt

In [33]:
!grep Box test.txt

                                <string><![CDATA[/Users/brianmapes/Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>


In [ ]:
template = '../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv'

In [17]:
gdf.time[i]

'2018-11-19T17:00:00'